<a href="https://colab.research.google.com/github/asyrofist/Simple-Traceability-SRS-Document/blob/main/modul_traceability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive

Mounted at /content/drive
/content/drive


In [ ]:
!pip install py-automl

# Preprocessing

In [30]:
import pandas as pd
import pandas as pd
import numpy as np
from spacy.lang.en import English
from tabulate import tabulate
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from pyAutoML.ml import ML,ml, EncodeCategorical
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

class prosesData:  
  def __init__(self, namaFile = '/content/drive/MyDrive/dataset/reqDataset_v1/dataset_baru.xlsx'):
      self.__dataFile = namaFile

  def fulldataset(self, inputSRS= 'SRS1'):
      xl = pd.ExcelFile(self.__dataFile)
      dfs = {sh:xl.parse(sh) for sh in xl.sheet_names}
      kalimat = dfs[inputSRS]
      kalimat_semua = kalimat.head(len(kalimat))
      return kalimat_semua

  def preprocessing(self):
      xl = pd.ExcelFile(self.__dataFile)
      for sh in xl.sheet_names:
        df = xl.parse(sh)
        print('Processing: [{}] ...'.format(sh))
        print(df.head())

  def apply_cleaning_function_to_list(self, X):
      cleaned_X = []
      for element in X:
          cleaned_X.append(prosesData.clean_text(self, raw_text= element))
      return cleaned_X

  def clean_text(self, raw_text):
      nlp = English()
      tokenizer = nlp.Defaults.create_tokenizer(nlp)
      tokens = tokenizer(raw_text)
      lemma_list = [token.lemma_.lower() for token in tokens if token.is_stop is False and token.is_punct is False and token.is_alpha is True]
      joined_words = ( " ".join(lemma_list))
      return joined_words  

  def __del__(self):
      print ('Destructor called.')    



class pengukuranEvaluasi:
  def __init__(self, dataPertama, dataKedua):
      self.data1 = dataPertama
      self.data2 = dataKedua

  def kmeans_cluster(self, nilai_cluster= 3):
      XVSM = np.array(self.data1)
      yVSM = np.array(self.data2)
      kmeans = KMeans(n_clusters=nilai_cluster) # You want cluster the passenger records into 2: Survived or Not survived
      kmeans.fit(XVSM)
      correct = 0
      for i in range(len(XVSM)):
          predict_me = np.array(XVSM[i].astype(float))
          predict_me = predict_me.reshape(-1, len(predict_me))
          prediction = kmeans.predict(predict_me)
          if prediction[0] == yVSM.all():
              correct += 1
      scaler = MinMaxScaler()
      XVSM_scaled = scaler.fit_transform(yVSM)
      print("data_correction {}".format(correct/len(XVSM)))
      return (XVSM_scaled)


  def ukur_evaluasi(self):
      X_train, X_test, y_train, y_test = train_test_split(pengukuranEvaluasi.kmeans_cluster(self), self.data2, test_size=0.3,random_state=109) # 70% training and 30% test
      y_train = y_train.argmax(axis= 1)
      X = X_train
      Y = y_train
      Y = EncodeCategorical(Y)
      size = 0.4
      return ML(X, Y, size, SVC(), RandomForestClassifier(), DecisionTreeClassifier(), KNeighborsClassifier(), LogisticRegression(max_iter = 7000))      

  def __del__(self):
      print ('Destructor called.')    


if __name__ == "__main__":
    myData = prosesData() # myData.preprocessing()
    req = myData.fulldataset() # myData.fulldataset(inputSRS)
    text_to_clean = list(req['Requirement Statement'])
    cleaned_text = myData.apply_cleaning_function_to_list(text_to_clean)
    data_raw = pd.DataFrame([text_to_clean, cleaned_text],index=['ORIGINAL','CLEANED'], columns= req['ID'])
    print(tabulate(data_raw, headers = 'keys', tablefmt = 'psql'))  
    myData.__del__()

    # myUkur= pengukuranEvaluasi(index1, index2)
    # myUkur.kmeans_cluster()
    # myUkur.ukur_evaluasi()


+----------+-------------------------------------------+---------------------------------------+-----------------------------------------------------+-----------------------------------------------+-------------------------------------------------------+-------------------------------------------------------------------+-----------------------------------------------+-----------------------------------+----------------------------------+--------------------------------------------------------+--------------------------------------------------------------------+----------------------------------------------------+------------------------------------------------------------------+
|          | F01                                       | F02                                   | F03                                                 | F04                                           | F05                                                   | F06                                                      

# Vector Space Model (VSM)

In [27]:
import numpy as np
import pandas as pd
import string #allows for format()
import math
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import pairwise_kernels
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler


class measurement:
  def __init__(selft):
      pass
  
  def bagOfWords(self, data_raw):
      b = CountVectorizer(data_raw) # dilakukan vektorisasi
      c = b.fit(data_raw) # dilakukan fiting 
      d = b.get_feature_names() # diambil namanya, sebagai kolom
      e = b.transform(data_raw).toarray() #data 
      f = req.ID # diambil sebagai indeks
      bow_df= pd.DataFrame(e, f, d) #data, indeks, kolom
      return bow_df

  def l2_normalizer(self, vec):
      denom = np.sum([el**2 for el in vec])
      return [(el / math.sqrt(denom)) for el in vec]

  def build_lexicon(self, corpus):
      lexicon = set()
      for doc in corpus:
          lexicon.update([word for word in doc.split()])
      return lexicon

  def freq(self, term, document):
    return document.split().count(term)

  def numDocsContaining(self, word, doclist):
      doccount = 0
      for doc in doclist:
          if measurement.freq(self, term= word, document= doc) > 0:
              doccount +=1
      return doccount 

  def idf(self, word, doclist):
      n_samples = len(doclist)
      df = measurement.numDocsContaining(self, word, doclist)
      return np.log(n_samples / 1+df)

  def build_idf_matrix(self, idf_vector):
      idf_mat = np.zeros((len(idf_vector), len(idf_vector)))
      np.fill_diagonal(idf_mat, idf_vector)
      return idf_mat

  def cosine_measurement(self, data):
      X = np.array(data)
      Y = np.array(data)
      cosine_similaritas = pairwise_kernels(X, Y, metric='linear')
      frequency_cosine = pd.DataFrame(cosine_similaritas, index=req['ID'],columns=req['ID'])
      return frequency_cosine    

  def threshold_value(self, threshold, data):
      dt = data.values >= threshold
      dt1 = pd.DataFrame(dt, index= data.index, columns= data.columns)
      mask = dt1.isin([True])
      dt3 = dt1.where(mask, other= 0)
      mask2 = dt3.isin([False])
      th_cosine1 = dt3.where(mask2, other= 1)
      return th_cosine1

  def __del__(self):
      print ('Destructor called.')    


if __name__ == "__main__":

      myVSMMeasurement = measurement()
      bow = myVSMMeasurement.bagOfWords(data_raw= cleaned_text)
      print("\nBag of Words")
      print(tabulate(bow, headers = 'keys', tablefmt = 'psql'))   

      mydoclist = cleaned_text
      vocabulary = myVSMMeasurement.build_lexicon(cleaned_text)

      # tfidf normal
      my_idf_vector = [myVSMMeasurement.idf(word, mydoclist) for word in vocabulary] # vektor idf
      my_idf_matrix = myVSMMeasurement.build_idf_matrix(my_idf_vector) # membuat matriks idf
      doc_term_matrix_tfidf = [np.dot(tf_vector, my_idf_matrix) for tf_vector in bow.values] 
      frequency_TFIDF_normal = pd.DataFrame(doc_term_matrix_tfidf, index= req.ID, columns= vocabulary) #hasil freq_tfidf
      print("\nTFIDF")
      print(tabulate(frequency_TFIDF_normal, headers = 'keys', tablefmt = 'psql'))   

      dt_cosine = myVSMMeasurement.cosine_measurement(frequency_TFIDF_normal.values)
      print("\ncosine + TFIDF with l2 normalizer")
      print(tabulate(dt_cosine, headers = 'keys', tablefmt = 'psql'))   

      print("\nthreshold normal")
      th_cosine = myVSMMeasurement.threshold_value(0.2, dt_cosine)   
      print(tabulate(myVSMMeasurement.threshold_value(0.2, dt_cosine), headers = 'keys', tablefmt = 'psql'))   

      # tfidf dengan l2 normalizer
      doc_term_matrix_l2 = [myVSMMeasurement.l2_normalizer(vec) for vec in bow.values]
      doc_term_matrix_tfidf_l2 = [myVSMMeasurement.l2_normalizer(tf_vector) for tf_vector in doc_term_matrix_tfidf]
      frequency_TFIDF_l2 = pd.DataFrame(doc_term_matrix_tfidf_l2, index= req.ID, columns= vocabulary) #hasil freq_tfidf
      print("\nTFIDF with l2 normalizer")
      print(tabulate(frequency_TFIDF_l2, headers = 'keys', tablefmt = 'psql'))   

      dt_cosine_l2 = myVSMMeasurement.cosine_measurement(frequency_TFIDF_l2.values)
      print("\ncosine + TFIDF with l2 normalizer")
      print(tabulate(dt_cosine_l2, headers = 'keys', tablefmt = 'psql'))   

      print("\nthreshold l2 normalizer")
      th_cosine_l2 = myVSMMeasurement.threshold_value(0.2, dt_cosine_l2)   
      print(tabulate(th_cosine_l2, headers = 'keys', tablefmt = 'psql')) 

      myVSMMeasurement.__del__()

      # myEvaluasi = pengukuranEvaluasi(dt_cosine_l2.values, th_cosine_l2.values) #ndengan l2 normalizer
      myEvaluasi = pengukuranEvaluasi(dt_cosine.values, th_cosine.values) # normal
      myEvaluasi.ukur_evaluasi()

Destructor called.

Bag of Words
+-------+-----------+------------+-------+-----------+---------------+----------------+---------+---------+----------+----------+-----------+-------------+----------+-----------+-----------+-------------+---------+----------+-----------+---------+------------+-----------+------------+------------+------------+-----------+-------------+---------------+------------+-----------+---------+----------+--------+----------+-----------+----------+--------+---------+--------+------------+---------+---------+
| ID    |   account |   accounts |   add |   android |   application |   applications |   asked |   based |   canvas |   choose |   confirm |   connected |   create |   desired |   devices |   different |   drawn |   friend |   friends |   gives |   internet |   message |   messages |   occupied |   portable |   receive |   recipient |   reliability |   requests |   results |   right |   search |   send |   sender |   senders |   system |   text |   types |  

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1

# Latent Semantic Analysis (LSA)

In [26]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from tabulate import tabulate


class latentSemantic:
  def __init__(self, data_raw= cleaned_text):
      self.__data = data_raw

  def ukurLSA(self):
      vectorizer = TfidfVectorizer(stop_words='english', 
                                    max_features= 1000, # keep top 1000 terms 
                                    max_df = 0.5, 
                                    smooth_idf=True)
      X = vectorizer.fit_transform(self.__data)
      svd_model = TruncatedSVD(n_components=len(self.__data), algorithm='randomized', n_iter=100, random_state=122)
      svd_model.fit(X)
      terms = vectorizer.get_feature_names()
      return pd.DataFrame(svd_model.components_, index= req.ID, columns= terms)

  def urutLSA(self):
      hasil_LSA = []
      for i, comp in enumerate(data_svd.components_):
          terms_comp = zip(data_terms, comp)
          sorted_terms = sorted(terms_comp, key= lambda x:x[1], reverse=True)[:7]
          for t in sorted_terms:
              hasil_LSA.append(t)
      return hasil_LSA

  def threshold_value(self, threshold, data):
        dt = data.values >= threshold
        dt1 = pd.DataFrame(dt, index= data.index, columns= data.columns)
        mask = dt1.isin([True])
        dt2 = dt1.where(mask, other= 0)
        mask2 = dt2.isin([False])
        tbl_5 = dt2.where(mask2, other= 1)
        return tbl_5

  def __del__(self):
      print ('Destructor called.')    

if __name__ == "__main__":
      myLSA = latentSemantic()
      dt_lsa = myLSA.ukurLSA()
      print(tabulate(dt_lsa, headers = 'keys', tablefmt = 'psql'))
      th_lsa = myLSA.threshold_value(0.2, dt_lsa)
      print(tabulate(dt_lsa, headers = 'keys', tablefmt = 'psql'))

      myEvaluasi = pengukuranEvaluasi(dataPertama= dt_lsa.values, dataKedua= th_lsa.values)
      myEvaluasi.ukur_evaluasi()

Destructor called.
+-------+--------------+--------------+--------------+--------------+---------------+----------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+---------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+
| ID    |      account |     accounts |          add |      android |   application |   applications |        asked |        based |       canvas |       choose |      confirm |    connected |       create |      desired |      devices |    different |        drawn |       friend |      friends |        gives |     internet |      message |     messa

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1

____________________________________________________
RandomForestClassifier ______________________________ 

Accuracy Score for RandomForestClassifier is 
0.0


Confusion Matrix for RandomForestClassifier is 
[[0 0 0 0 0]
 [0 0 1 0 0]
 [0 0 0 0 0]
 [0 0 1 0 0]
 [1 0 1 0 0]]


Classification Report for RandomForestClassifier is 
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       0.0
           1       0.00      0.00      0.00       1.0
           2       0.00      0.00      0.00       0.0
           4       0.00      0.00      0.00       1.0
           5       0.00      0.00      0.00       2.0

    accuracy                           0.00       4.0
   macro avg       0.00      0.00      0.00       4.0
weighted avg       0.00      0.00      0.00       4.0



____________________________________________________
DecisionTreeClassifier ______________________________ 

Accuracy Score for DecisionTreeClassifier is 
0.0


Confusion Matrix f

# Latent Dirichlet Allocation (LDA)

In [29]:
from time import time
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.datasets import fetch_20newsgroups

class latentDirichlet:
  def __init__(self, data_raw= cleaned_text):
      self.__data = data_raw
      self.__n_features = len(self.__data)

  def ukur_tfidf_vectorizer(self):
      # Use tf-idf features for NMF.
      tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2,
                                        max_features=self.__n_features,
                                        stop_words='english')
      tfidf = tfidf_vectorizer.fit_transform(self.__data)
      return tfidf_vectorizer, tfidf

  def ukur_tf(self):
      tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2,
                                      max_features=self.__n_features,
                                      stop_words='english')
      tf = tf_vectorizer.fit_transform(self.__data)
      return tf_vectorizer, tf

  def Frobenius_norm_feature(self):
      nmf = NMF(n_components=len(self.__data), random_state=1, alpha=.1, l1_ratio=.5).fit(latentDirichlet.ukur_tfidf_vectorizer(self)[1])
      nmf_tfidf = latentDirichlet.ukur_tfidf_vectorizer(self)[0].get_feature_names()
      fitur_frb_tfidf = (nmf_tfidf)
      data_frb_tfidf = (nmf.components_)
      dt_df =  pd.DataFrame(data_frb_tfidf, index= req.ID, columns= fitur_frb_tfidf)
      return dt_df

  def Kullback_feature(self):
      nmf = NMF(n_components=len(self.__data), random_state=1, beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1, l1_ratio=.5).fit(latentDirichlet.ukur_tfidf_vectorizer(self)[1])
      tfidf_feature_names = latentDirichlet.ukur_tfidf_vectorizer(self)[0].get_feature_names()
      fitur_kll_tfidfi = (tfidf_feature_names)
      data_kll_tfidf = (nmf.components_)
      dt_df =  pd.DataFrame(data_kll_tfidf, index= req.ID, columns= fitur_kll_tfidfi)
      return dt_df

  def lda_feature(self):
      lda = LatentDirichletAllocation(n_components=len(self.__data), max_iter=5, learning_method='online', learning_offset=50., random_state=0)
      lda.fit(latentDirichlet.ukur_tf(self)[1])
      tf_feature_names = latentDirichlet.ukur_tf(self)[0].get_feature_names()
      fitur_lda = (tf_feature_names)
      nmf = NMF(n_components=len(self.__data), random_state=1, beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1, l1_ratio=.5).fit(latentDirichlet.ukur_tfidf_vectorizer(self)[1])
      data_lda = (nmf.components_)
      dt_df =  pd.DataFrame(data_lda, index= req.ID, columns= fitur_lda)
      return dt_df

  def threshold_value(self, threshold, data):
      dt = data.values >= threshold
      dt1 = pd.DataFrame(dt, index= data.index, columns= data.columns)
      mask = dt1.isin([True])
      dt3 = dt1.where(mask, other= 0)
      mask2 = dt3.isin([False])
      th_cosine1 = dt3.where(mask2, other= 1)
      return th_cosine1

  def __del__(self):
      print ('Destructor called.')    


if __name__ == "__main__":
      myLDA = latentDirichlet()
      dt_fr = myLDA.Frobenius_norm_feature()
      print(tabulate(dt_fr, headers = 'keys', tablefmt = 'psql'))
      th_fr = myLDA.threshold_value(0.2, dt_fr)
      print(tabulate(th_fr, headers = 'keys', tablefmt = 'psql'))

      dt_kl = myLDA.Kullback_feature()
      print(tabulate(dt_kl, headers = 'keys', tablefmt = 'psql'))
      th_kl = myLDA.threshold_value(0.2, dt_kl)
      print(tabulate(th_kl, headers = 'keys', tablefmt = 'psql'))

      dt_lda = myLDA.lda_feature()
      print(tabulate(dt_lda, headers = 'keys', tablefmt = 'psql'))
      th_lda = myLDA.threshold_value(0.2, dt_lda)
      print(tabulate(th_lda, headers = 'keys', tablefmt = 'psql'))

      myLDA.__del__()

      # myEvaluasi = pengukuranEvaluasi(dataPertama= dt_lda.values, dataKedua= th_lda.values)
      # myEvaluasi = pengukuranEvaluasi(dataPertama= dt_kl.values, dataKedua= th_kl.values)
      myEvaluasi = pengukuranEvaluasi(dataPertama= dt_fr.values, dataKedua= th_fr.values)
      myEvaluasi.ukur_evaluasi()      

Destructor called.
+-------+-----------+----------+----------+----------+---------+------------+---------+
| ID    |   message |   search |     send |   sender |    user |   username |   users |
|-------+-----------+----------+----------+----------+---------+------------+---------|
| F01   |  0        |    0     | 0        | 0        | 0       |    0       | 0       |
| F02   |  0        |    0     | 0        | 0        | 1.20988 |    0       | 0       |
| F03   |  0        |    0     | 0        | 0        | 0       |    0       | 0       |
| F04   |  0.779288 |    0     | 0.819326 | 0.047748 | 0       |    0       | 0       |
| F05   |  0        |    0     | 0        | 0        | 0       |    0       | 0       |
| F06   |  0        |    1.096 | 0        | 0        | 0       |    0       | 0       |
| F07   |  0        |    0     | 0        | 0        | 0       |    1.13479 | 0       |
| NF01a |  0.186711 |    0     | 0        | 0.738124 | 0       |    0       | 0       |
| NF01b |  0 

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classificat